# 00_quickstart_qa_bot

This notebook follows a [tutorial](https://langchain.readthedocs.io/en/latest/modules/indexes/chain_examples/qa_with_sources.html) on how to build a bot that provides answers to questions using the `langchain` platform.

## 0. Install Packages and Modules

### 0.1 Install non-traditional packages

Packages that are required to run the notebook but are unlikely to be included in standard environments.

In [18]:
%%sh

pip install langchain
pip install openai
pip install PyPDF2
pip install nltk

[...]

_returning to this portion later to fill in the front matter_

In [19]:
import sys
import os

sys.path.append("../../../programs/.config/")

import openai_credentials

os.environ["OPENAI_API_KEY"] = openai_credentials.api_key

In [20]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

from langchain.indexes import VectorstoreIndexCreator

from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings

from langchain.text_splitter import CharacterTextSplitter

from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate

from langchain.llms import OpenAI

from PyPDF2 import PdfReader

In [6]:
reader = PdfReader(
    "../../../data/yellowstone/Lowenstern_Origins_Geothermal_Gases_at_Yellowstone.pdf"
)

lowenstern_text = [reader.pages[i].extract_text() for i in range(len(reader.pages))]
x = lowenstern_text[4:6]
lowenstern_text_reduced = " ".join(x)

In [5]:
lowenstern_text_reduced

'buried (trapped) thermal waters would then ﬂow along the base of lavas like the Elephant Back or Nez Perce ﬂows (Fig. 7ofHurwitz and Lowenstern, 2014) until they encounter the ground surface and emergeat low elevations (e.g., the Lower Geyser Basin). In contrast, steam pro-duced by deep boiling is able to rise along any existing fractures or per-meable volcanic vents to create acid sulfate terrains within and on top ofthe lava plateaus (e.g., Central Plateau, Smokejumper Hot Springs, Phan-tom Fumarole; all inFig. 2). Thus, the burial of previous thermal areasbeneath the Central Plateau Lavas enhanced the separation of steamand gas from residual thermal waters, aiding the formation of geochem-ically disparate acid sulfate and neutral chloride thermal areas.In the caldera, travertine-forming waters are found only at the coolperiphery of geothermal areas, such as near Firehole Lake, at HillsideSprings, and at Terrace Springs (Fournier, 1989). Outside the caldera, at the northern end of Y

In [21]:
# with open("../../../data/political/sotu_20230207.txt") as f:
#     sotu_text = f.read()

sotu_loader = TextLoader("../../../data/political/sotu_20230207.txt")
lowenstern_loader = TextLoader("../../../data/yellowstone/Lowenstern_Orrigins_Geothermal_Gases_at_Yellowstone.txt")

In [22]:
index = VectorstoreIndexCreator().from_loaders([lowenstern_loader])

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
Exiting: Cleaning up .chroma directory


In [25]:
query = "How old is the Yellowstone caldera?"
index.query_with_sources(query)

{'question': 'How old is the Yellowstone caldera?',
 'answer': ' The Yellowstone caldera is approximately 640,000 years old.\n',
 'sources': '../../../data/yellowstone/Lowenstern_Orrigins_Geothermal_Gases_at_Yellowstone.txt'}

In [24]:
query = "What is the dominant gas emitted from Yellowstone?"
index.query_with_sources(query)

{'question': 'What is the dominant gas emitted from Yellowstone?',
 'answer': ' The dominant gas emitted from Yellowstone is CO2.\n',
 'sources': '../../../data/yellowstone/Lowenstern_Orrigins_Geothermal_Gases_at_Yellowstone.txt'}

In [57]:
extra = []
for i in range(len(texts)):
    if len(texts[i]) > 1024:
        extra=+ i
        print(i)

1
3
5
7
9
11


In [82]:
type(sotu_text)

str

In [79]:
text_splitter = CharacterTextSplitter(chunk_size=750, chunk_overlap=0)
texts = text_splitter.split_text(sotu_text)

In [ ]:
embeddings = OpenAIEmbeddings()

In [80]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])
query = "What kind of problem is inflation"
docs = docsearch.similarity_search(query)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
Exiting: Cleaning up .chroma directory


In [87]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

chain({"input_documents": docs, "question": query}, return_only_outputs=True)

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 7843 tokens (7587 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.